In [1]:
import lotron2
import csv
import pandas as pd
import numpy as np
import os

In [6]:
bigwig_dir = '/project/davieslab/datashare/jhamley/cd4_enh/Z003/cd4_enh_Z003/'
oligo_file = bigwig_dir + 'cd4_hv3_hg38_enh2.txt'
write_dir = '/project/Wellcome_Discovery/datashare/lhentges/jhamley_mcc_cd4_enh/Z003/'
ext = '_ext_de_norm_rep_cd4_enh_Z003_merged.bw'


# lotron2 parameters
threshold_list = [2, 4, 6]
background_list = [10**4, 10**5, 10**6]
window_list = [200, 400, 600]
threshold_cumulative_init = 9
min_size = 50
max_size = 500
min_read_count = 5
single_read_cov_depth = 'estimate'

In [3]:
def calc_peak_stats(start, end, chrom_cov_array):
    # Calculate peak stats
    peak_size = end - start
    peak_max = chrom_cov_array[start:end].max()
    peak_max_pos = chrom_cov_array[start:end].argmax() + start
    peak_sum = chrom_cov_array[start:end].sum()
    peak_mean = chrom_cov_array[start:end].mean()

    return peak_size, peak_max, int(peak_max_pos), peak_sum, peak_mean    

In [7]:
def estimate_single_read_cov_depth(chrom_cov_array):
    chrom_cov_nonzero = np.ma.masked_where(chrom_cov_array <= 0, chrom_cov_array)
    min_reads = chrom_cov_nonzero.min()
    return min_reads

In [4]:
bigwig_list = []
for file in os.listdir(bigwig_dir):
    if file.endswith(ext):
        bigwig_list.append(bigwig_dir+file)

In [5]:
completed_list = []
for file in os.listdir(write_dir):
    if file.endswith('.csv'):
        completed_list.append(file.split('_')[0])

In [9]:
exp_dict = {}
with open(oligo_file) as f:
    csv_reader = csv.reader(f, delimiter='\t')
    for row in csv_reader:
        file = bigwig_dir + row[3] + ext
        if file in bigwig_list and row[3] not in completed_list:
            if row[3] not in exp_dict:
                exp_dict[row[3]] = {}
                exp_dict[row[3]]['chr'] = row[0]
                exp_dict[row[3]]['file'] = file
            else:
                print('duplicate', row[3])

In [10]:
print(len(exp_dict))

1


In [13]:
for i, exp in enumerate(exp_dict):
    print(i + 1, 'of', len(exp_dict), end='\r')
    bw_lotron2 = lotron2.BigwigData(exp_dict[exp]['file'])
    chrom_cov_array = bw_lotron2.get_chrom_info_make_coverage_map(exp_dict[exp]['chr'])
    if single_read_cov_depth == 'estimate':
        single_read_cov_depth_value = estimate_single_read_cov_depth(chrom_cov_array)
    else:
        single_read_cov_depth_value = single_read_cov_depth
    background_global_min = single_read_cov_depth_value * min_read_count
    mcc_peaks = bw_lotron2.call_candidate_peaks_lotron_chrom(threshold_cumulative_init, exp_dict[exp]['chr'], background_list, window_list, threshold_list, min_size, max_size, background_global_min)
    if len(mcc_peaks) == 0:
        print(exp, 'no peaks')
        continue
    mcc_peaks[['peak_size', 'peak_max', 'peak_max_pos', 'peak_sum', 'peak_mean']] = mcc_peaks.apply(lambda row: pd.Series(calc_peak_stats(row['Start'], row['End'], chrom_cov_array)), axis=1)
    mcc_peaks['peak_max_pos'] = mcc_peaks['peak_max_pos'].astype(int)
    mcc_peaks.sort_values(by=['peak_max'], inplace=True, ascending=False)
    mcc_peaks.to_csv(write_dir + exp + '_mcc_peaks.csv', index=False, header=True, sep='\t')


rs112534607 no peaks


In [12]:
mcc_peaks.head()

,Chromosome,Start,End


In [18]:
mcc_peaks.head()

,Chromosome,Start,End,peak_size,peak_max,peak_max_pos,peak_sum,peak_mean
1,chr5,130652118,130652442,324.0,235.632599,130652322,33716.483590,104.063221
4,chr5,130652565,130652636,71.0,139.067657,130652592,8332.810127,117.363523
3,chr5,130652510,130652533,23.0,91.930046,130652529,1836.248489,79.836891
5,chr5,130652766,130652769,3.0,31.575691,130652768,91.259457,30.419819
6,chr5,130653641,130654058,417.0,23.514524,130653895,6130.158633,14.700620


In [19]:
print(len(mcc_peaks))

7
